# Project Goal
- Create a predictive model that takes in a quarterback's past statistics to predict their __PFF offensive grade__ in the next season. PFF offensive grade is an aggregate measure of a quarterback's overall performance.

## Data Overview
- We have 18 seasons (2006-2023) of NFL Quarterback data acquired from [Pro Football Focus](https://www.pff.com/).
- This data is regular season only (excludes postseason games).
- Each season of passing data is contained in a csv file, as well as each rushing seasons. This is a total of 36 CSVs that we must combine.

# Load data
- Get all data into a single dataframe.
- Each row will represent a __single season's statistics__ from a single player.
- Most players will have multiple rows, as most played multiple seasons.

### Imports

In [2]:
import pandas as pd
import numpy as np
import os

from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from sklearn.preprocessing import StandardScaler

import matplotlib.pyplot as plt
%matplotlib inline

pd.options.display.max_columns = 100
pd.options.display.max_rows = 100

# global random_state
random_state = 9

In [3]:
# preview 2 QBs from 2006 season (passing stats)
sample_df = pd.read_csv('../pass_data/2006.csv')
sample_df.head(2)

,player,player_id,position,team_name,player_game_count,accuracy_percent,aimed_passes,attempts,avg_depth_of_target,avg_time_to_throw,bats,big_time_throws,btt_rate,completion_percent,completions,declined_penalties,def_gen_pressures,drop_rate,dropbacks,drops,first_downs,franchise_id,grades_hands_fumble,grades_offense,grades_pass,grades_run,hit_as_threw,interceptions,passing_snaps,penalties,pressure_to_sack_rate,qb_rating,sack_percent,sacks,scrambles,spikes,thrown_aways,touchdowns,turnover_worthy_plays,twp_rate,yards,ypa
0,Brett Favre,14,QB,GB,16,66.2,583,613,9.0,2.52,12,22,3.4,56.0,343,1,162,11.1,638,43,188,12,74.6,55.7,57.4,31.3,4,18,665,2,13.0,72.8,3.3,21,4,2,12,18,27,4.1,3904,6.4
1,Jon Kitna,217,QB,DET,16,71.1,560,596,9.3,2.79,9,24,3.8,62.4,372,0,245,6.5,677,26,221,11,62.0,64.3,65.5,43.9,6,22,714,3,25.7,79.9,9.3,63,18,2,19,21,33,4.6,4209,7.1


In [4]:
# preview 2 QBs from 2006 season (rushing stats)
sample_df = pd.read_csv('../rush_data/2006.csv').query("position == 'QB'")
sample_df.head(2)

,player,player_id,position,team_name,player_game_count,attempts,avoided_tackles,breakaway_attempts,breakaway_percent,breakaway_yards,declined_penalties,designed_yards,drops,elu_recv_mtf,elu_rush_mtf,elu_yco,elusive_rating,explosive,first_downs,franchise_id,fumbles,gap_attempts,grades_hands_fumble,grades_offense,grades_offense_penalty,grades_pass,grades_pass_block,grades_pass_route,grades_run,grades_run_block,longest,penalties,rec_yards,receptions,routes,run_plays,scramble_yards,scrambles,targets,total_touches,touchdowns,yards,yards_after_contact,yco_attempt,ypa,yprr,zone_attempts
43,Michael Vick,773,QB,ATL,16,50,17,6,10.8,112,1,280,0,0,5,85.0,16.7,44,55,2,9,11,43.8,66.5,64.9,57.8,61.3,60.0,84.8,60.1,51,3,1,1,1,448,759,73,1,51,2,1039,308,2.50,8.4,1.0,17
53,Vince Young,2948,QB,TEN,15,37,13,1,3.4,19,0,78,0,0,5,56.0,20.5,21,30,31,10,6,28.9,51.3,62.2,48.5,NaN,NaN,63.7,60.0,39,2,0,0,0,378,474,46,0,37,7,552,240,2.89,6.7,NaN,10


### Columns to drop

In [5]:
# drop unnecessary/redundant stats from passing data
pass_drops = ['player_id', 'position', 'declined_penalties', 'franchise_id', 'grades_run']

In [6]:
# drop unnecessary/redundant stats from rushing data. Also drop duplicate columns already contained in the passing data
rush_drops = ['player_id', 'position', 'team_name', 'player_game_count', 'declined_penalties', 'drops', 'franchise_id', 
              'grades_hands_fumble', 'grades_offense', 'grades_pass','grades_pass_block', 'grades_pass_route', 
              'grades_run_block', 'penalties', 'rec_yards', 'receptions', 'routes', 'scrambles', 'targets', 'yprr']

- Dropping features that won't impact on our predictions. This includes player_id, position (everyone is a QB) and franchise_id.
- Also dropping duplicate columns that appear in both the passing and rushing data.

### Passing data

In [7]:
# get all csv files in the 'pass_data' directory
file_paths = [os.path.join('../pass_data', file) for file in os.listdir('../pass_data') if file.endswith('.csv')]

# list to hold all 18 pass_dfs
pass_dfs = []

# read each file into a dataframe
for file_path in file_paths:
    
    # get year as string from filename
    year = file_path[-8:-4]
    
    # load each season into a df, get QBs, drop specified columns, set player name as index
    pass_data = pd.read_csv(file_path).query("position == 'QB'").drop(pass_drops, axis=1)
    
    # add 'pass' to cols to indicate passing stat
    pass_data.columns = ['player', 'team_name', 'player_game_count'] + ['pass_' + col for col in pass_data.columns[3:]]
    
    # add year column
    pass_data['year'] = year
    
    # add df to list
    pass_dfs.append(pass_data)

In [8]:
# look at 2006 and 2007 seasons
pass_dfs[0].shape, pass_dfs[1].shape

((70, 38), (81, 38))

- A single season of passing data contains 38 columns.
- The 2006 season has passing statistics for 70 unique QBs, the 2007 season has 81.

### Rushing data

In [10]:
# get all csv files in the 'rush_data' directory
file_paths = [os.path.join('../rush_data', file) for file in os.listdir('../rush_data') if file.endswith('.csv')]

# list to hold all 18 rush_dfs
rush_dfs = []

# read each file into a dataframe
for file_path in file_paths:
    
    # get year as string from filename
    year = file_path[-8:-4]
    
    # load each season into a df, get QBs, drop specified columns, set player name as index
    rush_data = pd.read_csv(file_path).query("position == 'QB'").drop(rush_drops, axis=1)
    
    # add 'rush' to cols to indicate rushing stat
    rush_data.columns = ['player'] + ['rush_' + col for col in rush_data.columns[1:]]
    
    # add year column
    rush_data['year'] = year
    
    # add df to list
    rush_dfs.append(rush_data)

In [11]:
# look at 2006 and 2007 seasons
rush_dfs[0].shape, rush_dfs[1].shape

((68, 28), (79, 28))

- A single season of rushing data contains 28 columns.
- The 2006 season has rushing statistics for 68 unique QBs, the 2007 season has 79.

### Merge pass and rush dfs

In [12]:
# stack 18 passing dataframes together
pass_df = pd.concat(pass_dfs, axis=0)

# duplicate players, unique QBs
pass_df.duplicated().sum(), pass_df['player'].nunique()

(0, 288)

- No duplicate players.
- 288 unique NFL quarterbacks.

In [13]:
# stack 18 rushing dataframes together
rush_df = pd.concat(rush_dfs, axis=0)

# duplicate players, unique QBs
rush_df = rush_df.drop_duplicates()
rush_df.duplicated().sum(), rush_df['player'].nunique()

(0, 277)

- No duplicate players.
- 275 unique NFL quarterbacks.

### Create master df

In [14]:
pass_df.shape, rush_df.shape

((1306, 38), (1254, 28))

In [15]:
# left join to ensure we have players with passing data, as this is more important than rushing data
df = pass_df.merge(rush_df, on=['player', 'year'], how='left')

df.shape, len(set(df.player))

((1306, 64), 288)

- 1306 QB seasons, 64 columns, 288 unique QBs.

# Passing TDs and Yds per game
- Adding this feature. Total touchdowns/yards are great indicators of performance, but normalizing them (dividing by games played) can avoid punishing players who get injured or miss games for other reasons.

In [17]:
# add column
df['pass_tds_gm'] = df['pass_touchdowns'] / df['player_game_count']

In [18]:
# add column
df['pass_yds_gm'] = df['pass_yards'] / df['player_game_count']

# Experience
- Adding a feature "exp" to represent the player's __years of experience__. For example, a rookie quarterback would have exp=0. 

### QBs who have 2006 data
- Most of these QBs are not rookies.
- Through external research, I manually found the experience of players in the 2006 season. 

In [19]:
# manual input found from research
qbs_2006_exp = [15, 10, 6, 8, 5, 2, 3, 6, 6, 3, 2, 11, 2, 4, 1, 14, 9, 2, 1, 4, 5, 0, 8, 0, 3, 0, 9, 7, 1, 13, 10, 4, 1, 13, 7, 
               7, 3, 12, 13, 3, 0, 7, 1, 3, 6, 5, 11, 0, 4, 3, 8, 5, 5, 8, 2, 13, 4, 3, 1, 5, 0, 1, 0, 19, 12, 7, 6, 3, 4, 0]

# append NaNs to fill out list for all non-2006 players
for _ in range(len(df) - len(qbs_2006_exp)):
    qbs_2006_exp.append(np.nan)

### Add experience col

In [20]:
# add experience col to df
df['exp'] = qbs_2006_exp

# check
df[['player', 'exp', 'year']]

,player,exp,year
0,Brett Favre,15.0,2006
1,Jon Kitna,10.0,2006
2,Marc Bulger,6.0,2006
3,Peyton Manning,8.0,2006
4,Drew Brees,5.0,2006
...,...,...,...
1301,C.J. Beathard,NaN,2023
1302,Sam Darnold,NaN,2023
1303,Mike White,NaN,2023
1304,Aaron Rodgers,NaN,2023


- QBs who played in 2006 season have an "exp" value, non-2006 QBs have NaN value.

In [21]:
# function to fill in experience column
def fill_experience(group):
    # get first experience value for a player
    first_exp = group['exp'].iloc[0]
    
    # if value is null, set to 0 (rookie season)
    if pd.isna(first_exp):
        first_exp = 0
    
    # define range of years to fill each player's experience column
    experience = range(int(first_exp), int(first_exp) + len(group))
    group['exp'] = list(experience)
    return group

# group by index and fill experience values, drop multi-index
df = df.groupby('player').apply(fill_experience).droplevel(0)

In [22]:
# check
df[df['player'] == 'Patrick Mahomes'][['player', 'year', 'exp']]

,player,year,exp
853,Patrick Mahomes,2017,0
875,Patrick Mahomes,2018,1
956,Patrick Mahomes,2019,2
1012,Patrick Mahomes,2020,3
1090,Patrick Mahomes,2021,4
1166,Patrick Mahomes,2022,5
1250,Patrick Mahomes,2023,6


- All players now have a corresponding experience for each of their seasons.

# Past performance
- Engineer two features that capture a QB's historical performance over their career (all past seasons and their current season).
- I will look at each player's offensive grade over their 3 most recent seasons, and use the mean (capturing peformance) along with the standard deviation (capturing consistency).

In [24]:
# ensure values are sorted properly
df = df.sort_values(by=['player', 'exp']).reset_index(drop=True)
df[['player', 'year', 'exp']].head()

,player,year,exp
0,A.J. Feeley,2006,5
1,A.J. Feeley,2007,6
2,A.J. Feeley,2011,7
3,A.J. McCarron,2015,0
4,A.J. McCarron,2017,1


In [25]:
# use current season's offensive grade + 2 previous seasons
window_size = 3

# calculate rolling mean and std
# 'min_periods=1' ensures we calculate statistics even if there's only one past season. drop multi-index with droplevel()
df['rolling_mean'] = df.groupby('player')['pass_grades_offense'].apply(
    lambda x: x.rolling(window=window_size, min_periods=1).mean()).droplevel(0)
df['rolling_std'] = df.groupby('player')['pass_grades_offense'].apply(
    lambda x: x.rolling(window=window_size, min_periods=1).std()).droplevel(0)

# handle NaN values
df['rolling_mean'].fillna(0, inplace=True)
df['rolling_std'].fillna(0, inplace=True)

In [27]:
# check
df[df['player'] == 'Patrick Mahomes'][['player', 'year', 'pass_grades_offense', 'rolling_mean', 'rolling_std']]

,player,year,pass_grades_offense,rolling_mean,rolling_std
993,Patrick Mahomes,2017,72.6,72.600000,0.000000
994,Patrick Mahomes,2018,93.2,82.900000,14.566400
995,Patrick Mahomes,2019,83.6,83.133333,10.307926
996,Patrick Mahomes,2020,91.4,89.400000,5.102940
997,Patrick Mahomes,2021,77.1,84.033333,7.159842
998,Patrick Mahomes,2022,91.3,86.600000,8.227393
999,Patrick Mahomes,2023,87.1,85.166667,7.294747


- Now we have two more engineered features that look at a QBs recent performance.
- Hopefully, this will give our models insight during the training process.

# Nulls

In [28]:
# look at null counts in each col
df.isna().sum().to_frame().T

,player,team_name,player_game_count,pass_accuracy_percent,pass_aimed_passes,pass_attempts,pass_avg_depth_of_target,pass_avg_time_to_throw,pass_bats,pass_big_time_throws,pass_btt_rate,pass_completion_percent,pass_completions,pass_def_gen_pressures,pass_drop_rate,pass_dropbacks,pass_drops,pass_first_downs,pass_grades_hands_fumble,pass_grades_offense,pass_grades_pass,pass_hit_as_threw,pass_interceptions,pass_passing_snaps,pass_penalties,pass_pressure_to_sack_rate,pass_qb_rating,pass_sack_percent,pass_sacks,pass_scrambles,pass_spikes,pass_thrown_aways,pass_touchdowns,pass_turnover_worthy_plays,pass_twp_rate,pass_yards,pass_ypa,year,rush_attempts,rush_avoided_tackles,rush_breakaway_attempts,rush_breakaway_percent,rush_breakaway_yards,rush_designed_yards,rush_elu_recv_mtf,rush_elu_rush_mtf,rush_elu_yco,rush_elusive_rating,rush_explosive,rush_first_downs,rush_fumbles,rush_gap_attempts,rush_grades_offense_penalty,rush_grades_run,rush_longest,rush_run_plays,rush_scramble_yards,rush_total_touches,rush_touchdowns,rush_yards,rush_yards_after_contact,rush_yco_attempt,rush_ypa,rush_zone_attempts,pass_tds_gm,pass_yds_gm,exp,rolling_mean,rolling_std
0,0,0,0,2,0,0,2,0,0,0,0,0,0,0,17,0,0,0,0,0,0,0,0,0,0,40,0,0,0,0,0,0,0,0,0,0,0,0,97,97,97,109,97,97,97,97,149,97,97,97,97,97,97,97,97,97,97,97,97,97,97,97,97,97,0,0,0,0,0


- Few nulls in the dataset.
- When looking at a players statistics, null values in these columns indicate 0 attempts for that specific statistic.
- These values can be filled with 0.

In [29]:
# fill nulls with value of 0
df = df.fillna(0)

# Target variable
- For each season __x__, the target is the grades_offense stat of season __x + 1__.

In [30]:
# add target col
df['target'] = df.groupby('player')['pass_grades_offense'].shift(-1)

# check
df.head(3)[['player', 'pass_grades_offense', 'target']]

,player,pass_grades_offense,target
0,A.J. Feeley,64.3,41.2
1,A.J. Feeley,41.2,50.0
2,A.J. Feeley,50.0,NaN


- Players in their final season will have a NaN value for the target variable (as seen above).
- These rows cannot be used to train model, as we don't have the grades_offense stat of the next year (it doesn't exist).
- This is handled in [this notebook](./models_1.ipynb).

In [31]:
# export data
df.to_csv('../train_data/data.csv')